In [8]:
# # Ajoute le dossier racine du projet au chemin d'import Python
# import sys
# from pathlib import Path

# project_root = Path("..").resolve()
# sys.path.append(str(project_root))

import sys
from pathlib import Path

# 🔧 Ajouter le dossier parent du notebook (où se trouve `metadata`) au path
sys.path.append(str(Path().resolve().parent))


In [12]:
import pandas as pd
from openai import OpenAI
import re
import unicodedata
from metadata.referentiels_multichoix import referentiels_multichoix
from metadata.new_column_names import new_column_names
import numpy as np
import json
from dotenv import load_dotenv
import os
from pydantic import BaseModel
from pathlib import Path



# Charger les variables depuis le fichier .env
load_dotenv()

# Clé API OpenAI
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = api_key)
# ==================== CHARGEMENT DES DONNEES ====================

# Définit le chemin absolu vers le fichier JSON
project_root = Path("..").resolve()
METADATA_DIR = project_root / "metadata"

def charger_referentiel_competences(path=METADATA_DIR / "referentiel_competences.json"):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def load_data(filepath="data/besoins_24_25.csv", n_rows=None):
    try:
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"Le fichier {filepath} n'existe pas")
        df = pd.read_csv(filepath, encoding="utf-8", sep=",", nrows=n_rows)
        if df.empty:
            raise pd.errors.EmptyDataError("Le fichier CSV est vide")
        return df
    except UnicodeDecodeError:
        raise ValueError(f"Erreur d'encodage du fichier {filepath}")
    except pd.errors.ParserError:
        raise ValueError(f"Format de fichier invalide: {filepath}")

def charger_referentiel_competences(path=METADATA_DIR / "referentiel_competences.json"):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# ==================== UTILS ====================
def nettoyage_general(df):
    """
    Nettoyage standard du DataFrame
    - Suppression des doublons
    - Suppression des espaces inutiles dans les colonnes texte
    - Nettoyage des points de suspension
    - Conversion des types (si possible)
    - Uniformisation des NaN
    """
    print("🔎 Nettoyage général en cours...")

    # 1. Retrait des doublons
    df = df.drop_duplicates()

    # 2. Nettoyage des colonnes texte
    for col in df.select_dtypes(include='object').columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.lower()
            .str.strip()
            .str.replace(r'\.{3,}', '', regex=True)  # suppression des "....."
            .replace('', np.nan)  # remplace les vides par NaN après nettoyage
            .str.replace(r"[^\w\sÀ-ÿ',-]", "", regex=True)  # suppression caractères spéciaux
        )


    # 3. Conversion numérique si possible
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except (ValueError, TypeError):
            pass  # colonne non convertible

    # 4. Uniformisation des valeurs manquantes
    df.replace(['None', 'nan', 'NaN', 'N/A', '', ' '], np.nan, inplace=True)

    print("✅ Nettoyage général terminé")
    return df


def normalize(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode().lower().strip()

def nettoyer_valeur(val):
    val = val.strip()
    val = re.sub(r'\s+', ' ', val)
    return val

def clean_theme(theme):
    theme = re.sub(r'^Aucun thème.*?["“”]', '', theme)
    theme = re.sub(r'["“”]', '', theme)
    return theme.strip().capitalize()

def theme_existe(theme, referentiel):
    theme_norm = normalize(theme)
    for officiel in referentiel:
        if normalize(officiel) == theme_norm:
            return officiel
    return None

# ==================== CLASSIFICATION GPT ====================

def classifier_avec_gpt(val, col, referentiels_multichoix,client):
    liste = referentiels_multichoix.get(col, [])

    prompt = f"""
Tu travailles pour le réseau OSUI - Mission Laïque Française.

Voici les réponses officielles possibles pour la question "{col}" :
{chr(10).join(f"- {v}" for v in liste) if liste else "(aucune liste connue)"}

Voici la réponse libre d'un enseignant : "{val}"

Ta tâche :
1. Si cette réponse correspond à un des thèmes officiels (même si l'orthographe, les majuscules ou l'ordre des mots sont différents), retourne exactement ce thème officiel.
2. Sinon, propose un nouveau thème court et pertinent.
3. Si la réponse est vide, hors sujet ou incompréhensible, retourne : **Autre (à clarifier)**

Ne donne qu'un seul thème. Aucune explication.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Tu es un expert en classification pédagogique."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1
    )

    theme_propose = response.choices[0].message.content.strip()
    theme_propre = clean_theme(theme_propose)

    officiel = theme_existe(theme_propre, liste)
    if officiel:
        return officiel

    INTERDITS = ["nouveau theme", "aucun theme", "veuillez fournir", "n/a", "non renseigné", "rien"]
    if any(bad in normalize(theme_propre) for bad in INTERDITS):
        return "Autre (à clarifier)"

    return theme_propre


# ==================== EVALUATION GPT ====================

# 🔧 Modèle de sortie typé pour GPT
class ScoreOnly(BaseModel):
    score: int  # Uniquement un score entre 0 et 3

# Fonction avec message system + message utilisateur
def evaluer_reponse_par_gpt_structured(reponse, competence, grille, client):
    """
    Utilise GPT-4o avec message system+user pour évaluer une réponse selon une grille de compétences.
    Retourne un score entier (0 à 3).
    """
    if not isinstance(reponse, str) or reponse.strip().lower() in ["nan", "none", ""]:
        return None

    # 🎓 Message système : définir le rôle et le comportement
    system_prompt = """
                    Tu es un responsable RH spécialisé en éducation, issu du modèle éducatif français.
                    Tu es chargé d'analyser des réponses d'enseignants et de leur attribuer un score de 0 à 3
                    en fonction d'une grille d'indicateurs de compétences professionnelles.
                    Tu dois répondre uniquement avec un objet JSON contenant un champ 'score', un entier entre 0 et 3.
                    Aucune justification n'est attendue.
                    """


    # 🧠 Message utilisateur : contenu spécifique à évaluer
    user_prompt =   f"""
                    Réponse de l'enseignant : "{reponse}"

                    Compétence évaluée : "{competence}"

                    Grille de positionnement :
                    0 - Non acquis : {grille['0']}
                    1 - En cours d'acquisition : {grille['1']}
                    2 - Acquis : {grille['2']}
                    3 - Expert : {grille['3']}

                    Attribue un score unique de 0 à 3 en fonction des descripteurs de la grille.
                    Réponds uniquement avec un objet JSON contenant le champ "score".
                    """

    try:
        response = client.beta.chat.completions.parse(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format=ScoreOnly
        )
        return response.choices[0].message.parsed.score
    except Exception as e:
        print("❌ Erreur GPT structuré :", e)
        return None

def ajouter_scores_skills(df, referentiels, client, skills_col):
    referentiels_filtrés = [item for item in referentiels if item["variable"] in skills_col]

    for item in referentiels_filtrés:
        var = item["variable"]
        if var not in df.columns:
            df[var + "_score"] = None
            continue
        df[var + "_score"] = df[var].apply(
            lambda r: evaluer_reponse_par_gpt_structured(r, item["competence"], item["grille"], client)
        )
    return df

# ==================== MAPPING ====================

def appliquer_mapping(text, mapping):
    if pd.isna(text):
        return ""
    result = set()
    for val in text.split(','):
        val = nettoyer_valeur(val)
        if val in mapping:
            result.add(mapping[val])
        elif normalize(val) in [normalize(v) for v in mapping]:
            for k in mapping:
                if normalize(k) == normalize(val):
                    result.add(mapping[k])
        else:
            result.add(val)
    return ",".join(result)

# ==================== TRAITEMENT ====================

def reponses_hors_referentiel(df, col, referentiels_multichoix):
    hors_ref = set()
    referentiel_normalise = {normalize(r) for r in referentiels_multichoix}
    for ligne in df[col].dropna().astype(str):
        for val in ligne.split(','):
            val = nettoyer_valeur(val)
            if val and normalize(val) not in referentiel_normalise:
                hors_ref.add(val)
    return hors_ref

def traiter_colonne_multichoix(df, col, referentiels_multichoix,client):
    print(f"\n🔧 Traitement : {col}")
    valeurs = reponses_hors_referentiel(df, col, referentiels_multichoix[col])
    print(f"🧹 {len(valeurs)} réponses hors référentiel détectées")

    themes_col = {}

    for val in sorted(valeurs):
        theme = classifier_avec_gpt(val, col, referentiels_multichoix,client)
        print(f"➡️ {val} → {theme}")
        themes_col[val] = theme

    nouveaux = set(themes_col.values())
    for t in nouveaux:
        if not theme_existe(t, referentiels_multichoix[col]) and t != "Autre (à clarifier)":
            referentiels_multichoix[col].append(t)

    df[col + '_themes'] = df[col].apply(lambda x: appliquer_mapping(x, themes_col))

    print(f"✅ {len(themes_col)} nouvelles valeurs mappées pour {col}")
    return df, None, themes_col

# ==================== SAUVEGARDE ====================

def sauvegarder_referentiel(referentiels_multichoix, path="metadata/referentiels_multichoix.py"):
    with open(path, "w", encoding="utf-8") as f:
        f.write("# Référentiel enrichi automatiquement\n")
        f.write("referentiels_multichoix = ")
        json.dump(referentiels_multichoix, f, ensure_ascii=False, indent=4)
    print(f"\n✅ Referentiels_multichoix sauvegardé dans : {path}")


In [13]:
skills_ref = charger_referentiel_competences()

# Liste des colonnes à traiter
multi_cols = [
    'certifications_educatives',
    'responsabilites_complementaires',
    'matieres_niveaux_actuels',
    'objectifs_outils_numeriques',
    'competences_dev_formations',
    'freins_formation_continue',
    'contenus_autoformation',
    'evaluations_alternatives'
]

skills_col = [
    "exemple_adaptation_enseignement",
    "conception_evaluation_suivi",
    "outil_numerique_benefices",
    "outils_suivi_apprentissage",
    "climat_classe",
    "adaptation_pratiques_specifiques",
    "lien_apprentissages_parcours",
    "projets_collaboration_impact",
    "implication_parents_suivi",
    "integration_contexte_etablissement"
]

skills_score_col = [col + "_score" for col in skills_col]


# ==================== PIPELINE ====================
def run_pipeline(filepath, new_column_names, referentiels_multichoix,
                 multi_cols, skills_col, skills_ref,client,
                 output_path="data/processed/processed_data.csv",
                 ref_path="metadata/referentiels_multichoix.py"):
    """
    Pipeline complet de traitement :
    - Chargement des données
    - Renommage des colonnes
    - Nettoyage général
    - Traitement multichoix
    - Sauvegarde du référentiel enrichi et du dataframe propre
    """
    print("\n🚀 Démarrage du pipeline de traitement...\n")

    # Chargement
    df = load_data(filepath,n_rows=100)
    df.rename(columns=new_column_names, inplace=True)

    # Nettoyage
    df = nettoyage_general(df)

    # Traitement multi-choix
    # all_dummies = []
    all_themes = {}
    for col in multi_cols:
        df, _, themes = traiter_colonne_multichoix(df, col, referentiels_multichoix,client)
        all_themes[col] = themes

    # traitement des skils
    df = ajouter_scores_skills(df, skills_ref, client, skills_col)

    # 🔥 Nettoyage final
    df = nettoyage_general(df)

    # Numérotation des enseignants
    df["enseignant_uid"] = ["E{:04d}".format(i) for i in range(1, len(df) + 1)]

    df.to_csv(output_path, index=False)
    sauvegarder_referentiel(referentiels_multichoix, path=ref_path)

    print("\n✅ Pipeline terminé avec succès.")
    return df, all_themes

In [14]:
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
METADATA_DIR = PROJECT_ROOT / "metadata"

filepath = PROJECT_ROOT / "data" / "raw" / "besoins_24_25.csv"
output_file = DATA_PROCESSED / "processed_data.csv"
ref_file = METADATA_DIR / "referentiels_multichoix.py"

df_clean, all_themes = run_pipeline(filepath,
                                    new_column_names,
                                    referentiels_multichoix,
                                    multi_cols, skills_col,
                                    skills_ref,
                                    client,
                                    output_path=output_file,
                                    ref_path=ref_file)



🚀 Démarrage du pipeline de traitement...

🔎 Nettoyage général en cours...
✅ Nettoyage général terminé

🔧 Traitement : certifications_educatives
🧹 30 réponses hors référentiel détectées
➡️ aucune → Aucune certification déclarée
➡️ bafa → Aucune certification déclarée
➡️ bnnsa → **autre (à clarifier)**
➡️ brevet d'état d'éducateur sportif → Diplôme sportif
➡️ c2i2e enseignant → **autre (à clarifier)**
➡️ capes → Concours de l'enseignement secondaire
➡️ certificat fle master → FLE
➡️ certification cinéma et audiovisuel → Cinéma et audiovisuel
➡️ cps maroc → Certification pédagogique spécifique au Maroc
➡️ daefle diplôme d'aptitudes à l'enseignement du fle alliance française → FLE
➡️ diplome de formateur psc secourisme → Diplôme sportif
➡️ du → Diplôme universitaire (DU)
➡️ enseignant de langue → Enseignement des langues
➡️ formation de formation jury delf jury cambridge → **autre (à clarifier)**
➡️ formatrice psc et gqs → **autre (à clarifier)**
➡️ inscription sur la liste d'aptitude à l